In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torchvision.transforms import ToTensor

In [8]:
flood_data = pd.read_csv("/content/datasets/kerala.csv")
df_cleaned = flood_data.dropna()

# Save the cleaned DataFrame to a new CSV file
df_cleaned.to_csv('/content/datasets/cleaned_data.csv', index=False)


In [ ]:
!pip install wandb

In [ ]:
import os
import wandb

# Paste your api key here
os.environ["WANDB_API_KEY"] = '5113c6e4fced8cbf6ed17e7bfd6402b76f3fc454'

wandb.init()
# Feel free to change these and experiment !!
config = wandb.config
config.learning_rate = 2e-5
config.batch_size = 32
config.output_size = 1
config.input_size = 1
config.hidden_size = 64
config.epochs = 80

wandb: Currently logged in as: fmbishu. Use `wandb login --relogin` to force relogin


In [9]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

class FloodDataset(Dataset):
    def __init__(self, data_file, transform=None):
        self.df = pd.read_csv(data_file)

        # Drop columns (make sure to assign the modified DataFrame back to self.df)
        self.df = self.df.drop(columns=[' ANNUAL RAINFALL', 'SUBDIVISION'])


        # Split dataframe into data and labels
        self.data = self.df.drop(columns=['FLOODS'])
        self.labels = self.df['FLOODS'].replace(['YES', 'NO'], [1, 0])

        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = self.data.iloc[index]
        label = self.labels.iloc[index]

        # Convert data and label to PyTorch tensors
        data_tensor = torch.tensor(item.values, dtype=torch.float32)
        label_tensor = torch.tensor(label, dtype=torch.float32)

        # Apply transformations if available
        if self.transform:
            data_tensor = self.transform(data_tensor)

        return data_tensor, label_tensor




transform = None
dataset = FloodDataset('/content/datasets/cleaned_data.csv', transform=transform)

# Split into training and testing sets
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

# Create DataLoader instances
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


In [27]:

x = flood_data.iloc[:,1:14]

x

,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,1901,28.7,44.7,51.6,160.0,174.7,824.6,743.0,357.5,197.7,266.9,350.8,48.4
1,1902,6.7,2.6,57.3,83.9,134.5,390.9,1205.0,315.8,491.6,358.4,158.3,121.5
2,1903,3.2,18.6,3.1,83.6,249.7,558.6,1022.5,420.2,341.8,354.1,157.0,59.0
3,1904,23.7,3.0,32.2,71.5,235.7,1098.2,725.5,351.8,222.7,328.1,33.9,3.3
4,1905,1.2,22.3,9.4,105.9,263.3,850.2,520.5,293.6,217.2,383.5,74.4,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,2014,4.6,10.3,17.9,95.7,251.0,454.4,677.8,733.9,298.8,355.5,99.5,47.2
114,2015,3.1,5.8,50.1,214.1,201.8,563.6,406.0,252.2,292.9,308.1,223.6,79.4
115,2016,2.4,3.8,35.9,143.0,186.4,522.2,412.3,325.5,173.2,225.9,125.4,23.6
116,2017,1.9,6.8,8.9,43.6,173.5,498.5,319.6,531.8,209.5,192.4,92.5,38.1


In [28]:
y = flood_data.iloc[:,-1].replace(['YES', 'NO'], [1, 0])
y

0      1
1      1
2      1
3      1
4      0
      ..
113    1
114    0
115    0
116    0
117    1
Name: FLOODS, Length: 118, dtype: int64

In [29]:
x_train, x_test = train_test_split(x, test_size=0.2, random_state=42)
y_train, y_test = train_test_split(y, test_size=0.2, random_state=42)

In [ ]:
# import torchvision.transforms as transforms
# from torchvision import datasets

# training_x = torch.tensor(x_train.values, dtype=torch.float32)
# testing_x = torch.tensor(x_test.values, dtype=torch.float32)
# training_y = torch.tensor(y_train.values, dtype=torch.float32)
# testing_y = torch.tensor(y_test.values, dtype=torch.float32)

# train_data =FloodDataset(training_data,transform= transforms.ToTensor())

In [ ]:
# from torch.utils.data.dataloader import DataLoader
# from torch.utils import data
# #dataset chunk to use for each iteration
batchsize = 32

# train_dataloader = DataLoader(training_x, batch_size= batchsize)
# test_dataloader = DataLoader(testing_x, batch_size = batchsize)

In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
import torch.nn as nn

class Distra(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(Distra, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        if x.dim() == 0:
            raise ValueError("Input tensor has no dimensions")

        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))

        # Use the output from the last time step
        out_last_step = out[:, -1, :]

        # Adjust the Linear layer input size based on the output of the LSTM
        out_fc = self.fc(out_last_step)

        return out_fc



In [16]:
input_size = 13
hidden_size = 64
num_layers = 2
output_size = 1
model = Distra(input_size, hidden_size, num_layers, output_size)
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


In [17]:
# the training function
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch,( X, y) in enumerate(dataloader):
        X, y =  X.to(device).unsqueeze(0), y.to(device).unsqueeze(0)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        #Zero_grad  sens the gradient to zero after every iteration
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [18]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device).unsqueeze(0), y.to(device).unsqueeze(0)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [19]:

epochs = 80
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, loss_fn, optimizer)
    test(test_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 32])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 30])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 24])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(

loss: 0.558863  [    1/   94]
Test Error: 
 Accuracy: 41.7%, Avg loss: 0.687935 

Epoch 2
-------------------------------
loss: 0.509704  [    1/   94]
Test Error: 
 Accuracy: 41.7%, Avg loss: 0.643226 

Epoch 3
-------------------------------
loss: 0.452200  [    1/   94]
Test Error: 
 Accuracy: 41.7%, Avg loss: 0.607885 

Epoch 4
-------------------------------
loss: 0.495193  [    1/   94]
Test Error: 
 Accuracy: 41.7%, Avg loss: 0.592914 

Epoch 5
-------------------------------
loss: 0.579142  [    1/   94]
Test Error: 
 Accuracy: 41.7%, Avg loss: 0.582750 

Epoch 6
-------------------------------
loss: 0.397192  [    1/   94]
Test Error: 
 Accuracy: 41.7%, Avg loss: 0.555437 

Epoch 7
-------------------------------
loss: 0.432441  [    1/   94]
Test Error: 
 Accuracy: 41.7%, Avg loss: 0.545367 

Epoch 8
-------------------------------
loss: 0.392132  [    1/   94]
Test Error: 
 Accuracy: 41.7%, Avg loss: 0.536485 

Epoch 9
-------------------------------
loss: 0.377450  [    1/ 

In [20]:
#save model
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [21]:

model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [41]:
import torch

model.eval()

x = torch.tensor(x_test.head(1).values, dtype=torch.float32).unsqueeze(0)
y = torch.tensor(y_test.head(1).values, dtype=torch.float32).unsqueeze(0)
print(y_test.head(1))

with torch.no_grad():
    predict = model(x)


threshold = 0.4
predicted_labels = (predict >= threshold).int()

predicted_label = predicted_labels.item()

print(predicted_label)


56    1
Name: FLOODS, dtype: int64
0


In [44]:
import torch

# Load the model file

checkpoint = torch.load('model.pth')

# Check the contents of the checkpoint
print(checkpoint)


OrderedDict([('lstm.weight_ih_l0', tensor([[-0.0185,  0.1083,  0.0066,  ..., -0.0925,  0.1008,  0.0097],
        [ 0.1172, -0.1174,  0.0175,  ..., -0.0311,  0.0486,  0.0611],
        [ 0.0050, -0.1239,  0.0707,  ...,  0.0739,  0.0268, -0.0422],
        ...,
        [-0.0535,  0.1187, -0.0762,  ..., -0.0562,  0.0404, -0.0475],
        [-0.1226,  0.0685,  0.1010,  ..., -0.0937, -0.0571, -0.0424],
        [-0.1052, -0.0638, -0.0781,  ..., -0.0879,  0.0701, -0.1246]])), ('lstm.weight_hh_l0', tensor([[ 0.0050, -0.0155, -0.1230,  ...,  0.0220, -0.1132,  0.1196],
        [ 0.1173, -0.0155, -0.0242,  ...,  0.1177, -0.0299, -0.0244],
        [-0.0314,  0.0327, -0.0557,  ..., -0.1142, -0.0268,  0.0127],
        ...,
        [ 0.0140, -0.0014,  0.0556,  ...,  0.0460, -0.0572, -0.0610],
        [ 0.1176,  0.0490, -0.1025,  ...,  0.1218, -0.0426,  0.0141],
        [ 0.1204, -0.0518,  0.0760,  ...,  0.0381, -0.0939,  0.0650]])), ('lstm.bias_ih_l0', tensor([ 0.0446, -0.0130,  0.0079, -0.0121,  0.0746